In this notebook, you will learn how to make your first submission to the [Tabular Playground Series - Mar 2021 competition.](https://www.kaggle.com/c/tabular-playground-series-mar-2021)

# Make the most of this notebook!

You can use the "Copy and Edit" button in the upper right of the page to create your own copy of this notebook and experiment with different models. You can run it as is and then see if you can make improvements.

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
        
input_path = Path('/kaggle/input/tabular-playground-series-mar-2021/')

# Read in the data files

In [ ]:
train= pd.read_csv(input_path / 'train.csv', index_col='id')
display(train.head())


In [ ]:
test= pd.read_csv(input_path / 'test.csv', index_col='id')
display(test.head())

In [ ]:
target = train['target'].values


In [ ]:
columns = test.columns[1:]
columns

In [ ]:
cont_cols = [col for col in columns if 'cont' in col]
cat_cols = [col for col in columns if 'cat' in col]

def label_encode(train_df, test_df, column):
    le = LabelEncoder()
    new_feature = "{}_le".format(column)
    le.fit(train_df[column].unique().tolist() + test_df[column].unique().tolist())
    train_df[new_feature] = le.transform(train_df[column])
    test_df[new_feature] = le.transform(test_df[column])
    return new_feature

le_cols = []
for feature in cat_cols:
    le_cols.append(label_encode(train, test, feature))
    
columns = cont_cols + le_cols

In [ ]:
def run_rskf(train, target, clf, params):
    train_preds = np.zeros((train.shape[0], 2))
    test_preds = 0
    rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1337)
    for fold, (train_index, val_index) in enumerate(rskf.split(train, target)):
        print("-> Fold {}".format(fold + 1))
       
        x_train, x_valid = train.iloc[train_index][columns], train.iloc[val_index][columns]
        y_train, y_valid = target[train_index], target[val_index]
    
        model = clf(**params)
        model.fit(x_train, y_train,
                    eval_set=[(x_valid, y_valid)], 
                    verbose=0,
                    early_stopping_rounds=500)
        train_oof_preds = model.predict_proba(x_valid)[:,1]
        train_preds[val_index, fold//5] = train_oof_preds
        test_oof_preds = model.predict_proba(test[columns])[:,1]
        test_preds += test_oof_preds / 10
        print("ROC AUC Score = {}".format(roc_auc_score(y_valid, train_oof_preds)))
        if fold in [4, 9]:
            print("=> Overall ROC AUC Score = {}".format(roc_auc_score(target, train_preds[:, fold//5])))
    return model, test_preds

In [ ]:
params_lgb = {#'device':'gpu',
            #'random_state':2021,
            #'cat_l2':25.999876242730252,
            'cat_smooth':89.2699690675538,
            'colsample_bytree':0.2557260109926193,
             #'early_stopping_round':200,
            'learning_rate':0.00918685483594994,
            'max_bin':788,
            'max_depth':81,
            'metric':"auc",
            'min_child_samples':292,
            'min_data_per_group':177,
            'n_estimators':16000,
            'n_jobs':-1,
            'num_leaves':171,
            'reg_alpha':0.7115353581785044,
            'reg_lambda':5.658115293998945,
            'subsample':0.9262904583735796,
            'subsample_freq':1,
            'verbose':-1}

clf_lgb = LGBMClassifier

In [ ]:
model_lgb, test_preds_lgb = run_rskf(train, target, clf_lgb , params_lgb)

In [ ]:
submission = pd.read_csv(input_path / 'sample_submission.csv')
display(submission.head())

In [ ]:

submission['target'] = test_preds_lgb
submission.to_csv('submission_lgbm2.csv', index=False)

## Now you should save your Notebook (blue button in the upper right), and then when that's complete go to the notebook viewer and make a submission to the competition. :-)

## There's lots of room for improvement. What things can you try to get a better score?